In [0]:
init_load_flag=int(dbutils.widgets.get("init_load_flag"))

### Reading the table silver customers

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
df=spark.sql("select * from catabricks_cat.silver.customers_silver")

### Removing Duplicates


In [0]:
df=df.dropDuplicates(["customer_id"])

### Dividing old vs new records

In [0]:
if init_load_flag==0:
    df_old=spark.sql('''select DimCustomerKey,customer_id,create_date,update_date from catabricks_cat.gold.DimCustomers''')
else:
    df_old=spark.sql('''select 0 DimCustomerKey,0 customer_id,0 create_date,0 update_date from catabricks_cat.silver.customers_silver where 1=0''')

### Reaniming columns of df_old

In [0]:
df_old=df_old.withColumnRenamed("DimCustomerKey","old_DimCustomerKey")\
    .withColumnRenamed("customer_id","old_customer_id")\
    .withColumnRenamed("create_date","old_create_date")\
    .withColumnRenamed("update_date","old_update_date")

### Appyling join with the old records with df


In [0]:
df_join=df.join(df_old,on=df.customer_id==df_old['old_customer_id'],how='left')

### Seperating old vs new records

In [0]:
df_new=df_join.filter(col('old_DimCustomerKey').isNull())

In [0]:
df_old=df_join.filter(col('old_DimCustomerKey').isNotNull())    

### Preparing df_old

In [0]:
# Dropping all the columns which are not required
df_old=df_old.drop('old_customer_id','old_update_date')
# Renaming old_DimCustomerKey
df_old=df_old.withColumnRenamed("old_DimCustomerKey","DimCustomerKey")
# Renaming old_create_date column to create_date
df_old=df_old.withColumnRenamed("old_create_date","create_date")
df_old=df_old.withColumn("create_date",to_timestamp(col("create_date")))
# Recreating "update_date" column with current timestamp
df_old=df_old.withColumn("update_date",current_timestamp())


### Preparing df_new

In [0]:
# Dropping all the columns which are not required
df_new=df_new.drop('old_DimCustomerKey','old_customer_id','old_update_date','old_create_date')
# Recreating "create_date" column with current timestamp
df_new=df_new.withColumn("create_date",current_timestamp())
# Recreating "update_date" column with current timestamp
df_new=df_new.withColumn("update_date",current_timestamp())

In [0]:
df_new=df_new.withColumn("DimCustomerKey",monotonically_increasing_id()+lit(1))

In [0]:
df_new.display()

### Adding max surrogate key

In [0]:
df_new=df_new.withColumn("DimCustomerKey",monotonically_increasing_id()+lit(1))

In [0]:
if init_load_flag==1:
    max_surrogate_key=0

else :
    df_maxsur=spark.sql('''select max(DimCustomerKey) as max_surrogate_key from catabricks_cat.gold.DimCustomers''')
    ### Converting df_maxsur to max_surrogate key variable
    max_surrogate_key=df_maxsur.collect()[0]['max_surrogate_key'] 

In [0]:
df_new=df_new.withColumn('DimCustomerKey',lit(max_surrogate_key)+col('DimCustomerKey')) 

In [0]:
df_new.display()

### Union of df_new and df_old

In [0]:
df_final=df_new.unionByName(df_old)

In [0]:
df_final.display()

### SCD type 1

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("catabricks_cat.gold.DimCustomers"):
    dlt_obj= DeltaTable.forPath(spark,"abfss://gold@databricksstorageak.dfs.core.windows.net/DimCustomers")
    dlt_obj.alias("trg").merge(df_final.alias("src"),"trg.DimCustomerKey=src.DimCustomerKey").whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()

else :
    df_final.write.mode("overwrite")\
    .option("path","abfss://gold@databricksstorageak.dfs.core.windows.net/DimCustomers").saveAsTable("catabricks_cat.gold.DimCustomers")




In [0]:
%sql
select * from catabricks_cat.gold.DimCustomers